In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from datasets import load_dataset, load_metric
import pandas as pd
from evaluate import load
import statistics

In [3]:
# !pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 42.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 42.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:000:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=16ec1199092785e5f61584d3ac167a6e93166d23db223b62696425464ea47b1e
  Stored in directory: /cephfs/home/startseva/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
# from sentence_transformers import SentenceTransformer

# sent_transf = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
# def get_embeddings(sentences):
#     embeddings = []
#     for sent in sentences:
#         embeddings.append(sent_transf.encode(sent))
#     return embeddings

In [62]:
df = pd.read_csv('dialogues_with_len_2_12.csv')
df

,conv_id,utterance_idx,personality,history,history_len,random_len,len_2,len_4,len_6,len_8,...,dialogue_8,dialogue_10,dialogue_12,target_2,target_4,target_6,target_8,target_10,target_12,persona
0,0,7,"['I read twenty books a year.', ""I'm a stunt d...","['Hello what are doing today?', 'I am good, I ...",15,12,"['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...",...,HUMAN: Hello what are doing today?\nPERSON: I ...,HUMAN: Hello what are doing today?\nPERSON: I ...,HUMAN: Hello what are doing today?\nPERSON: I ...,"I am good, I just got off work and tired, I ha...","I rather read, I've read about 20 books this y...",But a good movie is always good.,I work in the movies as well.,"Yes it is neat, I stunt double, it is so much ...","Nice, I only have one parent so now I help out...",I read twenty books a year. I'm a stunt double...
1,1,6,"['I have two dogs.', 'I like to work on vintag...","['Hi! how are you doing tonight?', ""I'm doing ...",13,2,"['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...",...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,I'm doing great. Just relaxing with my two dogs.,That's neat. What kind of volunteer work do yo...,Good for you. Do you like vintage cars? I've t...,Cars are my thing. Vintage cars. I love workin...,I didn't think about the crowd aspect of wrest...,I agree. Working out is a great way to burn of...,I have two dogs. I like to work on vintage car...
2,2,6,"['I love disneyland and mickey mouse.', 'I lov...","['Hello, how are you doing?', 'I love spending...",13,2,"['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...",...,"HUMAN: Hello, how are you doing?\nPERSON: I lo...","HUMAN: Hello, how are you doing?\nPERSON: I lo...","HUMAN: Hello, how are you doing?\nPERSON: I lo...",I love spending time with my family.,So then have you ever been to disneyland?,Yes I love mickey mouse such a cute little rat.,Man your lucky I am a baby delivery nurse and ...,I have to walk 3 miles to work to save money e...,Your economic skills are amazing.,I love disneyland and mickey mouse. I love to ...
3,3,6,"['I love to drink fancy tea.', 'I have a big l...",['Hi how are you doing? I am okay how about yo...,13,12,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,...,HUMAN: Hi how are you doing? I am okay how abo...,HUMAN: Hi how are you doing? I am okay how abo...,HUMAN: Hi how are you doing? I am okay how abo...,I am doing good. Just sipping tea. What do you...,I am sorry to hear that. What happened.,So what do you do now for fun? I like to read.,What kind of dogs do you have?,I love reading. I have a big library at my house.,I've a tough time hearing so reading is my hap...,I love to drink fancy tea. I have a big librar...
4,4,6,"[""I'm a little girl."", ""I've superpowers."", 'I...","['Hi I dye my hair 4 times a year.', 'Do you l...",13,6,"['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...",...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,Do you live in the sky? I do. in the clouds.,I make time stop. I've a superpower. I'm a sup...,I love living in the

In [63]:
df_pers = df[['conv_id', 'persona']].copy()
df_pers

,conv_id,persona
0,0,I read twenty books a year. I'm a stunt double...
1,1,I have two dogs. I like to work on vintage car...
2,2,I love disneyland and mickey mouse. I love to ...
3,3,I love to drink fancy tea. I have a big librar...
4,4,I'm a little girl. I've superpowers. I like to...
...,...,...
995,995,My husband is adopted. I've a german shepherd ...
996,996,I produce music for artists. I drive a 2015 ho...
997,997,I'm terrified of scorpions. I'm employed by th...
998,998,I am a musician and hope to make it big some d...


In [64]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [65]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [66]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [95]:
def get_embeds(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    encoded_input.to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    
    return sentence_embeddings.detach().cpu().numpy()

In [70]:
def make_list(str_pers):
    list_pers = str_pers.split('. ')
    for i in range(len(list_pers) - 1):
        list_pers[i] += '.'
    return list_pers

In [71]:
make_list("I read twenty books a year. I'm a stunt double as my second job. I only eat kosher. I was raised in a single parent household.")

['I read twenty books a year.',
 "I'm a stunt double as my second job.",
 'I only eat kosher.',
 'I was raised in a single parent household.']

In [72]:
df_pers['persona_list'] = df_pers['persona'].apply(make_list)
df_pers

,conv_id,persona,persona_list
0,0,I read twenty books a year. I'm a stunt double...,"[I read twenty books a year., I'm a stunt doub..."
1,1,I have two dogs. I like to work on vintage car...,"[I have two dogs., I like to work on vintage c..."
2,2,I love disneyland and mickey mouse. I love to ...,"[I love disneyland and mickey mouse., I love t..."
3,3,I love to drink fancy tea. I have a big librar...,"[I love to drink fancy tea., I have a big libr..."
4,4,I'm a little girl. I've superpowers. I like to...,"[I'm a little girl., I've superpowers., I like..."
...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[My husband is adopted., I've a german shepher..."
996,996,I produce music for artists. I drive a 2015 ho...,"[I produce music for artists., I drive a 2015 ..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[I'm terrified of scorpions., I'm employed by ..."
998,998,I am a musician and hope to make it big some d...,[I am a musician and hope to make it big some ...


In [73]:
df_pers['embeds'] = df_pers['persona_list'].apply(get_embeds)
df_pers

,conv_id,persona,persona_list,embeds
0,0,I read twenty books a year. I'm a stunt double...,"[I read twenty books a year., I'm a stunt doub...","[[0.07642344, -0.0124058565, 0.016273646, 0.04..."
1,1,I have two dogs. I like to work on vintage car...,"[I have two dogs., I like to work on vintage c...","[[0.0019214519, -0.038681567, 0.074155144, 0.0..."
2,2,I love disneyland and mickey mouse. I love to ...,"[I love disneyland and mickey mouse., I love t...","[[0.033331506, -0.036470093, 0.053791195, -0.0..."
3,3,I love to drink fancy tea. I have a big librar...,"[I love to drink fancy tea., I have a big libr...","[[-0.04142527, -0.023192937, 0.054251615, 0.02..."
4,4,I'm a little girl. I've superpowers. I like to...,"[I'm a little girl., I've superpowers., I like...","[[0.069716774, 0.017751405, 0.012737408, 0.058..."
...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[My husband is adopted., I've a german shepher...","[[-0.06767419, 0.043871682, 0.07021701, 0.0672..."
996,996,I produce music for artists. I drive a 2015 ho...,"[I produce music for artists., I drive a 2015 ...","[[0.01843293, -0.036188986, -0.0134823425, 0.0..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[I'm terrified of scorpions., I'm employed by ...","[[0.027389033, -0.00783816, -0.010162611, 0.10..."
998,998,I am a musician and hope to make it big some d...,[I am a musician and hope to make it big some ...,"[[0.055158496, -0.0051588663, 0.050338022, -0...."


In [74]:
df_pers['embeds'][0]

array([[ 0.07642344, -0.01240586,  0.01627365, ..., -0.07298425,
        -0.08684872, -0.08259922],
       [-0.03847928, -0.0291817 ,  0.05053766, ..., -0.08003151,
        -0.09196994, -0.03010596],
       [ 0.06239585, -0.00590506,  0.01726064, ..., -0.07025286,
         0.01402865, -0.06130337],
       [-0.00817018,  0.04261496,  0.01058122, ...,  0.02794386,
        -0.01597427,  0.00267592]], dtype=float32)

In [106]:
def get_embeds_out(df, i):
    df_i = pd.read_csv(f'output_pers_dial_length_{i}.csv')
    df_i.fillna('', inplace=True)
    df[f'embed_{i}'] = df_i['pred'].apply(get_embeds)
    return df

In [107]:
for i in range(2, 13, 2):
    df_pers = get_embeds_out(df_pers, i)
df_pers

,conv_id,persona,persona_list,embeds,embed_2,embed_4,embed_6,embed_8,embed_10,embed_12
0,0,I read twenty books a year. I'm a stunt double...,"[I read twenty books a year., I'm a stunt doub...","[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[-0.02765868, -0.13093577, 0.04948815, 0.0601...","[[-0.0039304937, -0.11751381, -0.012045448, 0....","[[-0.01759668, -0.006890785, 0.12005855, 0.057...","[[-0.016534202, -0.03744092, -0.0044592475, -0...","[[-0.038459547, -0.028786002, -0.0006414625, -...","[[0.0379851, -0.032014858, -0.0045721205, 0.00..."
1,1,I have two dogs. I like to work on vintage car...,"[I have two dogs., I like to work on vintage c...","[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[-0.067996405, -0.024530565, 0.006690696, -0....","[[-0.037459213, 0.0014372907, 0.065871045, 0.0...","[[0.02984807, 0.031104624, 0.027298342, 0.0553...","[[0.089583725, -0.02144081, 0.10484952, -0.033...","[[0.0688171, -0.051169567, 0.076147534, -0.031...","[[-0.052663855, -0.010014518, 0.0154790105, 0...."
2,2,I love disneyland and mickey mouse. I love to ...,"[I love disneyland and mickey mouse., I love t...","[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.014700983, -0.06987183, 0.060451612, -0.0...","[[0.00851145, -0.0550255, 0.0015292238, -0.004...","[[-0.053678416, -0.060420953, 0.11003696, -0.0...","[[-0.011269063, 0.008178735, -0.064653635, -0....","[[-0.0649343, 0.032825086, -0.03042778, -0.021...","[[-0.0987668, 0.004687645, -0.0049891644, -0.0..."
3,3,I love to drink fancy tea. I have a big librar...,"[I love to drink fancy tea., I have a big libr...","[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[0.001710127, -0.02236993, 0.07673711, 0.0157...","[[-0.033290613, -0.0448982, -0.013805023, 0.02...","[[0.016978262, -0.095082365, 0.03173648, 0.069...","[[0.024956252, -0.04190829, 0.05379579, 0.0873...","[[0.022539783, -0.06778806, 0.023187714, 0.042...","[[-0.013982476, 0.043272212, 0.028534971, 0.04..."
4,4,I'm a little girl. I've superpowers. I like to...,"[I'm a little girl., I've superpowers., I like...","[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[0.028118223, -0.016325897, 0.027416863, 0.03...","[[-0.052945815, -0.048917625, 0.0055377367, 0....","[[0.020471243, -0.0033110399, 0.06367538, 0.01...","[[0.009226361, -0.04631145, 0.04349037, -0.003...","[[-0.029578365, -0.06045842, -0.017077103, -0....","[[-0.010345881, -0.011278771, -0.009968636, 0...."
...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[My husband is adopted., I've a german shepher...","[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[0.027539134, -0.008092664, 0.091679946, 0.04...","[[-0.0332499, 0.051869817, -0.026154028, 0.080...","[[0.072018676, -0.043807026, 0.06544037, -0.00...","[[0.020897616, -0.058173135, 0.063586235, 0.02...","[[0.04450335, -0.022425108, 0.042618223, 0.060...","[[0.0009950027, 0.05576224, 0.11649566, 0.0151..."
996,996,I produce music for artists. I drive a 2015 ho...,"[I produce music for artists., I drive a 2015 ...","[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[0.039550778, -0.056519493, 0.03819225, 0.058...","[[-0.044982284, -0.032547235, -0.016049258, -0...","[[-0.04751811, 0.029016914, 0.01207492, 0.0441...","[[0.038930934, -0.027818657, 0.045437243, 0.05...","[[0.020824084, -0.09306747, 0.037195098, 0.078...","[[-0.056663033, 0.017945955, 0.018328315, 0.04..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[I'm terrified of scorpions., I'm employed by ...","[[0.027389033, -0.00783816, -0.010162611, 0.10...","[[0.0372995, -0.023412624, 0.0036345753, -0.05...","[[0.024111882, 0.011764647, 0.021427356, 0.027...","[[0.023163764, 0.02891762, -0.002524299, 0.031...","[[0.08331697, -0.024241785, 0.034295633, -0.00...","[[0.017075205, -0.025039677, -0.0064893975, 0....","[[0.010801233, -0.02251149, 0.025832333, 0.041..."
998,998,I am a musician and hope to make it big some d...,[I am a musician and hope to make it bi

In [110]:
df_pers['embeds'][0].shape

(4, 384)

In [123]:
from scipy.spatial import distance
import numpy as np

In [124]:
def map_pred_pers(embeds_pers, embed_pred):
    dists = []
    for pers_i in embeds_pers:
        dists.append(distance.cosine(pers_i.reshape(-1), embed_pred.reshape(-1)))
        
    score = np.min(np.array(dists))
    return score

In [125]:
df_eval = df_pers.copy()

for i in range(2, 13, 2):
    df_eval[f'score_{i}'] = df_eval.apply(lambda x: map_pred_pers(x['embeds'], x[f'embed_{i}']), axis=1)
df_eval

,conv_id,persona,persona_list,embeds,embed_2,embed_4,embed_6,embed_8,embed_10,embed_12,score_2,score_4,score_6,score_8,score_10,score_12
0,0,I read twenty books a year. I'm a stunt double...,"[I read twenty books a year., I'm a stunt doub...","[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[-0.02765868, -0.13093577, 0.04948815, 0.0601...","[[-0.0039304937, -0.11751381, -0.012045448, 0....","[[-0.01759668, -0.006890785, 0.12005855, 0.057...","[[-0.016534202, -0.03744092, -0.0044592475, -0...","[[-0.038459547, -0.028786002, -0.0006414625, -...","[[0.0379851, -0.032014858, -0.0045721205, 0.00...",0.738688,0.903937,0.923750,0.801600,0.578763,0.464254
1,1,I have two dogs. I like to work on vintage car...,"[I have two dogs., I like to work on vintage c...","[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[-0.067996405, -0.024530565, 0.006690696, -0....","[[-0.037459213, 0.0014372907, 0.065871045, 0.0...","[[0.02984807, 0.031104624, 0.027298342, 0.0553...","[[0.089583725, -0.02144081, 0.10484952, -0.033...","[[0.0688171, -0.051169567, 0.076147534, -0.031...","[[-0.052663855, -0.010014518, 0.0154790105, 0....",0.819110,0.757341,0.842326,0.780913,0.495530,0.825590
2,2,I love disneyland and mickey mouse. I love to ...,"[I love disneyland and mickey mouse., I love t...","[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.014700983, -0.06987183, 0.060451612, -0.0...","[[0.00851145, -0.0550255, 0.0015292238, -0.004...","[[-0.053678416, -0.060420953, 0.11003696, -0.0...","[[-0.011269063, 0.008178735, -0.064653635, -0....","[[-0.0649343, 0.032825086, -0.03042778, -0.021...","[[-0.0987668, 0.004687645, -0.0049891644, -0.0...",0.731459,0.728925,0.461717,0.835417,0.465182,0.793972
3,3,I love to drink fancy tea. I have a big librar...,"[I love to drink fancy tea., I have a big libr...","[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[0.001710127, -0.02236993, 0.07673711, 0.0157...","[[-0.033290613, -0.0448982, -0.013805023, 0.02...","[[0.016978262, -0.095082365, 0.03173648, 0.069...","[[0.024956252, -0.04190829, 0.05379579, 0.0873...","[[0.022539783, -0.06778806, 0.023187714, 0.042...","[[-0.013982476, 0.043272212, 0.028534971, 0.04...",0.764319,0.405561,0.698177,0.821049,0.661772,0.727027
4,4,I'm a little girl. I've superpowers. I like to...,"[I'm a little girl., I've superpowers., I like...","[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[0.028118223, -0.016325897, 0.027416863, 0.03...","[[-0.052945815, -0.048917625, 0.0055377367, 0....","[[0.020471243, -0.0033110399, 0.06367538, 0.01...","[[0.009226361, -0.04631145, 0.04349037, -0.003...","[[-0.029578365, -0.06045842, -0.017077103, -0....","[[-0.010345881, -0.011278771, -0.009968636, 0....",0.839076,0.622011,0.687447,0.799768,0.643794,0.718703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[My husband is adopted., I've a german shepher...","[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[0.027539134, -0.008092664, 0.091679946, 0.04...","[[-0.0332499, 0.051869817, -0.026154028, 0.080...","[[0.072018676, -0.043807026, 0.06544037, -0.00...","[[0.020897616, -0.058173135, 0.063586235, 0.02...","[[0.04450335, -0.022425108, 0.042618223, 0.060...","[[0.0009950027, 0.05576224, 0.11649566, 0.0151...",0.728434,0.581269,0.646806,0.480219,0.668127,0.915387
996,996,I produce music for artists. I drive a 2015 ho...,"[I produce music for artists., I drive a 2015 ...","[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[0.039550778, -0.056519493, 0.03819225, 0.058...","[[-0.044982284, -0.032547235, -0.016049258, -0...","[[-0.04751811, 0.029016914, 0.01207492, 0.0441...","[[0.038930934, -0.027818657, 0.045437243, 0.05...","[[0.020824084, -0.09306747, 0.037195098, 0.078...","[[-0.056663033, 0.017945955, 0.018328315, 0.04...",0.833476,0.853916,0.574145,0.734435,0.913565,0.695060
997,997,I'm terrified of scorpions. I'm employed by th...,"[I'm terrified of scorpions., I'm employed by ...","[[0.0273

In [138]:
df_eval.to_csv('cosine_scores_for_diff_len.csv')

In [128]:
for i in range(2, 13, 2):
    print(statistics.mean(df_eval[f'score_{i}']))

0.7995725759398192
0.7282963635344059
0.7020065282378346
0.7005794412456453
0.7028836884498596
0.7110795717858709


In [133]:
names = ['output_task_pers_dial_random_1.csv',
        'output_zeroshot_random_1.csv',
        'output_pers_dial_random_1.csv',
        'output_cherry_pers_dial_random_1.csv',
        'output_cherry_task_pers_dial_random_1.csv',
        'output_random_eg_pers_dial_1.csv',
        'output_task_random_eg_pers_dial_1.csv'
        ]

In [132]:
df_pers_2 = df_pers[['conv_id', 'persona', 'embeds']].copy()
df_pers_2

,conv_id,persona,embeds
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04..."
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0..."
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0..."
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02..."
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058..."
...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672..."
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[[0.027389033, -0.00783816, -0.010162611, 0.10..."
998,998,I am a musician and hope to make it big some d...,"[[0.055158496, -0.0051588663, 0.050338022, -0...."


In [136]:
def get_embeds_out_names(df, name, i):
    df_i = pd.read_csv(name)
    df_i.fillna('', inplace=True)
    df[f'embed_{i}'] = df_i['pred'].apply(get_embeds)
    return df

In [139]:
for i in range(len(names)):
    df_pers_2 = get_embeds_out_names(df_pers_2, names[i], i)
df_pers_2

,conv_id,persona,embeds,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[-0.055509098, -0.12709782, 0.042557284, -0.0...","[[0.016230289, 0.027109366, -0.04994221, 0.045...","[[-0.02027111, -0.023154657, 0.08823594, 0.045...","[[0.026874265, 0.048068933, -0.027285332, 0.03...","[[-0.026324285, 0.050918788, 0.028246501, -0.0...","[[0.072135754, 0.11262077, 0.028331926, 0.0410...","[[0.04406476, -0.02947644, 0.03983019, 0.02747..."
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[-0.024029007, -0.053931337, 0.06669516, 0.03...","[[-0.11883839, 0.048298743, -0.0025480925, -0....","[[-0.1459724, 0.06425122, -0.00972668, 0.05378...","[[-0.023529043, 0.050771497, 0.07013742, 0.027...","[[0.006838527, -0.02276926, 0.13443793, 0.0718...","[[-0.008378779, -0.034078844, 0.026690977, 0.0...","[[-0.042750057, 0.008766829, 0.034629725, 0.00..."
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.062729456, -0.08481334, 0.07127799, -0.00...","[[0.0156225655, 0.03832794, 0.051564515, 0.065...","[[-0.08649873, -0.08295998, -0.050633494, -0.0...","[[0.0011914427, 0.0065149846, 0.040607482, 0.0...","[[0.010044505, -0.0665162, 0.040511206, 0.0391...","[[-0.011148175, -0.024961192, 0.08295765, -0.0...","[[-0.040324852, -0.039118476, 0.1258079, 0.060..."
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[-0.069078796, -0.16239387, 0.03922492, 0.042...","[[0.037078593, -0.08539534, 5.4909746e-05, 0.0...","[[0.019623816, 0.071917765, -0.011255553, 0.07...","[[0.057937197, -0.10252689, 0.020841992, 0.011...","[[0.07272652, -0.061152305, 0.052646663, 0.015...","[[-0.061021224, -0.13571283, -0.0023117498, -0...","[[0.009939846, -0.0035748072, 0.102498576, 0.0..."
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[-0.02941016, -0.083113045, 0.052923847, -0.0...","[[-0.059335407, -0.05404556, 0.02638905, 0.052...","[[-0.009203707, -0.07843648, 0.040802207, -0.0...","[[0.035085272, 0.01947357, -0.0029511652, 0.00...","[[0.003836091, -0.023455212, -0.09764387, 0.04...","[[-0.007305424, -0.09160913, -0.048722457, 0.0...","[[0.050015327, 0.06071533, 0.05651262, -0.0279..."
...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[0.0013726867, -0.013768116, 0.032212052, 0.0...","[[-0.03339534, -0.084973685, 0.039001573, 0.02...","[[-0.030166997, -0.020877773, 0.031442188, -0....","[[-0.03828201, -0.03598075, -0.056403186, -0.0...","[[0.0052572265, -0.10993585, 0.030037967, 0.02...","[[0.043987777, -0.06212764, 0.10049955, 0.0275...","[[0.059325173, -0.057326972, 0.01352955, -0.01..."
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[-0.0007710142, -0.030147612, 0.030434141, -0...","[[-0.06380042, -0.04645809, -0.016490519, 0.07...","[[0.028417079, -0.08404938, 0.0671451, -0.0196...","[[-0.012016205, -0.063385636, -0.010943645, 0....","[[-0.0476349, 0.0062611084, -0.029278772, 0.03...","[[-0.037264977, 0.020136205, 0.044046782, -0.0...","[[0.008306461, -0.105624884, 0.07049021, -0.09..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[[0.027389033, -0.00783816, -0.010162611, 0.10...","[[-0.018756485, 0.007650729, 0.017120846, -0.0...","[[-0.012534823, -0.043477938, 0.028908702, 0.0...","[[0.002313058, -0.04392178, 0.03177681, -0.002...","[[0.019236106, -0.060914613, -0.0063705584, -0...","[[-0.027077278, -0.11382563, 0.008449723, 0.01...","[[0.11889834, -0.07279743, 0.00401314, 0.06840...","[[0.03625243, 0.00044168739, 0.11167259, 0.106..."
998,998,I am a musician and hope to make it big some d...,"[[0.055158496, -0.0051588663, 0.050338022, -0

In [140]:
df_eval_2 = df_pers_2.copy()

for i in range(len(names)):
    df_eval_2[f'score_{i}'] = df_eval_2.apply(lambda x: map_pred_pers(x['embeds'], x[f'embed_{i}']), axis=1)
df_eval_2

,conv_id,persona,embeds,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,score_0,score_1,score_2,score_3,score_4,score_5,score_6
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[-0.055509098, -0.12709782, 0.042557284, -0.0...","[[0.016230289, 0.027109366, -0.04994221, 0.045...","[[-0.02027111, -0.023154657, 0.08823594, 0.045...","[[0.026874265, 0.048068933, -0.027285332, 0.03...","[[-0.026324285, 0.050918788, 0.028246501, -0.0...","[[0.072135754, 0.11262077, 0.028331926, 0.0410...","[[0.04406476, -0.02947644, 0.03983019, 0.02747...",0.657607,0.867164,0.578122,0.761785,0.832993,0.813389,0.663896
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[-0.024029007, -0.053931337, 0.06669516, 0.03...","[[-0.11883839, 0.048298743, -0.0025480925, -0....","[[-0.1459724, 0.06425122, -0.00972668, 0.05378...","[[-0.023529043, 0.050771497, 0.07013742, 0.027...","[[0.006838527, -0.02276926, 0.13443793, 0.0718...","[[-0.008378779, -0.034078844, 0.026690977, 0.0...","[[-0.042750057, 0.008766829, 0.034629725, 0.00...",0.863606,0.881549,0.853990,0.784940,0.529424,0.848716,0.776355
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.062729456, -0.08481334, 0.07127799, -0.00...","[[0.0156225655, 0.03832794, 0.051564515, 0.065...","[[-0.08649873, -0.08295998, -0.050633494, -0.0...","[[0.0011914427, 0.0065149846, 0.040607482, 0.0...","[[0.010044505, -0.0665162, 0.040511206, 0.0391...","[[-0.011148175, -0.024961192, 0.08295765, -0.0...","[[-0.040324852, -0.039118476, 0.1258079, 0.060...",0.776642,0.777908,0.824521,0.761767,0.868626,0.671153,0.729582
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[-0.069078796, -0.16239387, 0.03922492, 0.042...","[[0.037078593, -0.08539534, 5.4909746e-05, 0.0...","[[0.019623816, 0.071917765, -0.011255553, 0.07...","[[0.057937197, -0.10252689, 0.020841992, 0.011...","[[0.07272652, -0.061152305, 0.052646663, 0.015...","[[-0.061021224, -0.13571283, -0.0023117498, -0...","[[0.009939846, -0.0035748072, 0.102498576, 0.0...",0.585281,0.557051,0.707351,0.579061,0.261468,0.630431,0.733755
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[-0.02941016, -0.083113045, 0.052923847, -0.0...","[[-0.059335407, -0.05404556, 0.02638905, 0.052...","[[-0.009203707, -0.07843648, 0.040802207, -0.0...","[[0.035085272, 0.01947357, -0.0029511652, 0.00...","[[0.003836091, -0.023455212, -0.09764387, 0.04...","[[-0.007305424, -0.09160913, -0.048722457, 0.0...","[[0.050015327, 0.06071533, 0.05651262, -0.0279...",0.749051,0.175567,0.686374,0.877060,0.960985,0.694929,0.661472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[0.0013726867, -0.013768116, 0.032212052, 0.0...","[[-0.03339534, -0.084973685, 0.039001573, 0.02...","[[-0.030166997, -0.020877773, 0.031442188, -0....","[[-0.03828201, -0.03598075, -0.056403186, -0.0...","[[0.0052572265, -0.10993585, 0.030037967, 0.02...","[[0.043987777, -0.06212764, 0.10049955, 0.0275...","[[0.059325173, -0.057326972, 0.01352955, -0.01...",0.751982,0.439595,0.523972,0.877552,0.783756,0.530231,0.723252
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[-0.0007710142, -0.030147612, 0.030434141, -0...","[[-0.06380042, -0.04645809, -0.016490519, 0.07...","[[0.028417079, -0.08404938, 0.0671451, -0.0196...","[[-0.012016205, -0.063385636, -0.010943645, 0....","[[-0.0476349, 0.0062611084, -0.029278772, 0.03...","[[-0.037264977, 0.020136205, 0.044046782, -0.0...","[[0.008306461, -0.105624884, 0.07049021, -0.09...",0.610271,0.824759,0.346769,0.621760,0.648364,0.680881,0.644944
997,997,I'm terrified of scorpions. I'm employed by t

In [144]:
for i in range(len(names)):
    print(names[i], statistics.mean(df_eval_2[f'score_{i}']))

output_task_pers_dial_random_1.csv 0.6711482362560928
output_zeroshot_random_1.csv 0.7291331279575824
output_pers_dial_random_1.csv 0.6883179323859513
output_cherry_pers_dial_random_1.csv 0.6945349921276793
output_cherry_task_pers_dial_random_1.csv 0.6762235726397484
output_random_eg_pers_dial_1.csv 0.6887171045746654
output_task_random_eg_pers_dial_1.csv 0.6747162303831429


In [145]:
df_eval_2.to_csv('cosine_scores_for_prompts.csv')

# Targets

In [147]:
df_pers_3 = df_pers[['conv_id', 'persona', 'embeds']].copy()
df_pers_3

,conv_id,persona,embeds
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04..."
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0..."
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0..."
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02..."
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058..."
...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672..."
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0..."
997,997,I'm terrified of scorpions. I'm employed by th...,"[[0.027389033, -0.00783816, -0.010162611, 0.10..."
998,998,I am a musician and hope to make it big some d...,"[[0.055158496, -0.0051588663, 0.050338022, -0...."


In [149]:
def get_embeds_out_names(df, name, i):
    df_i = pd.read_csv(name)
    df_i.fillna('', inplace=True)
    df[f'embed_{i}'] = df_i['target'].apply(get_embeds)
    return df

In [150]:
for i in range(len(names)):
    df_pers_3 = get_embeds_out_names(df_pers_3, names[i], i)
df_pers_3

,conv_id,persona,embeds,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015..."
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594..."
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00..."
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0...."
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052..."
...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0..."
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0...."
997,997,I'm terrified of scorpions. I'm employed by th...,"[[0.027389033, -0.00783816, -0.010162611, 0.10...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02...","[[0.0064910064, 0.005406583, 0.059073173, 0.02..."
998,998,I am a musician and hope to make it big some d...,"[[0.055158496, -0.0051588663, 0.050338022, -0

In [151]:
df_eval_3 = df_pers_3.copy()

for i in range(len(names)):
    df_eval_3[f'score_{i}'] = df_eval_3.apply(lambda x: map_pred_pers(x['embeds'], x[f'embed_{i}']), axis=1)
df_eval_3

,conv_id,persona,embeds,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,score_0,score_1,score_2,score_3,score_4,score_5,score_6
0,0,I read twenty books a year. I'm a stunt double...,"[[0.07642344, -0.0124058565, 0.016273646, 0.04...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...","[[0.013898608, -0.08630135, 0.028049277, 0.015...",0.422401,0.422401,0.422401,0.422401,0.422401,0.422401,0.422401
1,1,I have two dogs. I like to work on vintage car...,"[[0.0019214519, -0.038681567, 0.074155144, 0.0...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...","[[0.046624925, -0.07575615, 0.10269241, 0.0594...",0.436309,0.436309,0.436309,0.436309,0.436309,0.436309,0.436309
2,2,I love disneyland and mickey mouse. I love to ...,"[[0.033331506, -0.036470093, 0.053791195, -0.0...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...","[[-0.033588022, 0.046847902, 0.034389846, 0.00...",0.029643,0.029643,0.029643,0.029643,0.029643,0.029643,0.029643
3,3,I love to drink fancy tea. I have a big librar...,"[[-0.04142527, -0.023192937, 0.054251615, 0.02...","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....","[[0.057576247, -0.039801463, -0.011354893, -0....",0.625194,0.625194,0.625194,0.625194,0.625194,0.625194,0.625194
4,4,I'm a little girl. I've superpowers. I like to...,"[[0.069716774, 0.017751405, 0.012737408, 0.058...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...","[[-0.012113794, 0.038446672, 0.06190131, 0.052...",0.428455,0.428455,0.428455,0.428455,0.428455,0.428455,0.428455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,My husband is adopted. I've a german shepherd ...,"[[-0.06767419, 0.043871682, 0.07021701, 0.0672...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...","[[-0.009916351, 0.077159196, -0.0006081593, -0...",0.910658,0.910658,0.910658,0.910658,0.910658,0.910658,0.910658
996,996,I produce music for artists. I drive a 2015 ho...,"[[0.01843293, -0.036188986, -0.0134823425, 0.0...","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....","[[0.049775593, -0.0122350715, -0.028304374, 0....",0.278181,0.278181,0.278181,0.278181,0.278181,0.278181,0.278181
997,997,I'm terrified of scorpions. I'm employed by t

In [152]:
for i in range(len(names)):
    print(names[i], statistics.mean(df_eval_3[f'score_{i}']))

output_task_pers_dial_random_1.csv 0.5506829087436199
output_zeroshot_random_1.csv 0.5506829087436199
output_pers_dial_random_1.csv 0.5506829087436199
output_cherry_pers_dial_random_1.csv 0.5506829087436199
output_cherry_task_pers_dial_random_1.csv 0.5506829087436199
output_random_eg_pers_dial_1.csv 0.5506829087436199
output_task_random_eg_pers_dial_1.csv 0.5506829087436199


In [153]:
df_eval_3.to_csv('cosine_scores_for_targets.csv')

# FineD-Eval

In [155]:
import pandas as pd

In [160]:
import random
import numpy as np
from datasets import load_dataset

In [161]:
dataset = load_dataset("bavard/personachat_truecased")
df_train = pd.DataFrame(dataset['train'])
df_valid = pd.DataFrame(dataset['validation'])
df_valid

No config specified, defaulting to: personachat_truecased/full
Found cached dataset personachat_truecased (/home/startseva/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638)


  0%|          | 0/2 [00:00<?, ?it/s]

,personality,candidates,history,conv_id,utterance_idx
0,"[I read twenty books a year., I'm a stunt doub...",[Oh really? I am actually in high school and I...,[Hello what are doing today?],0,0
1,"[I read twenty books a year., I'm a stunt doub...",[Why have you not sent help?! The scorpions ar...,"[Hello what are doing today?, I am good, I jus...",0,1
2,"[I read twenty books a year., I'm a stunt doub...","[Oh I'm sure they are., That would be great! W...","[Hello what are doing today?, I am good, I jus...",0,2
3,"[I read twenty books a year., I'm a stunt doub...","[That is great! Are you going to college?, Ugh...","[Hello what are doing today?, I am good, I jus...",0,3
4,"[I read twenty books a year., I'm a stunt doub...",[Big city life to overwhelming too much pressu...,"[Hello what are doing today?, I am good, I jus...",0,4
...,...,...,...,...,...
7796,"[I am a vegan and I love hummus., I love rolle...","[Haha really, that s the truth., Nope! I am a ...","[Hi how are you?, I am doing great on this cri...",999,3
7797,"[I am a vegan and I love hummus., I love rolle...","[Do you watch sports together?, Are you in jai...","[Hi how are you?, I am doing great on this cri...",999,4
7798,"[I am a vegan and I love hummus., I love rolle...",[Yes let us get to know each other bit that is...,"[Hi how are you?, I am doing great on this cri...",999,5
7799,"[I am a vegan and I love hummus., I love rolle...",[He's excited. He buys me all sorts of books t...,"[Hi how are you?, I am doing great on this cri...",999,6


In [162]:
df_dialogues = df_valid.groupby('conv_id').agg({'utterance_idx': 'max', 'personality': 'last', 'history': 'last'})
df_dialogues

,utterance_idx,personality,history
conv_id,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus..."
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre..."
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti..."
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live..."
...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good..."
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...


In [163]:
df_dialogues['history_len'] = df_dialogues['history'].apply(lambda x: len(x))
df_dialogues

,utterance_idx,personality,history,history_len
conv_id,,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus...",15
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre...",13
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti...",13
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...,13
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live...",13
...,...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...,13
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good...",15
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...,15


In [164]:
np.random.seed(42)
random.seed(42)

df_dialogues['random_len'] = df_dialogues['history_len'].apply(lambda x: random.randrange(2, x + 1, 2))
df_dialogues['random_len'].value_counts()

12    156
8     155
4     152
2     145
10    145
6     141
14    105
16      1
Name: random_len, dtype: int64

In [165]:
df_dialogues

,utterance_idx,personality,history,history_len,random_len
conv_id,,,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus...",15,12
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre...",13,2
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti...",13,2
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...,13,12
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live...",13,6
...,...,...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...,13,12
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good...",15,8
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...,15,8


In [166]:
rand_dialogs = df_dialogues.copy()
rand_dialogs

,utterance_idx,personality,history,history_len,random_len
conv_id,,,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus...",15,12
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre...",13,2
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti...",13,2
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...,13,12
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live...",13,6
...,...,...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...,13,12
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good...",15,8
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...,15,8


In [167]:
def cut_list(utts, needed_len):
    return utts[:needed_len]

In [168]:
rand_dialogs['cutted_history'] = rand_dialogs.apply(lambda x: cut_list(x['history'], x['random_len']), axis=1)
rand_dialogs

,utterance_idx,personality,history,history_len,random_len,cutted_history
conv_id,,,,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus...",15,12,"[Hello what are doing today?, I am good, I jus..."
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre...",13,2,"[Hi! how are you doing tonight?, I'm doing gre..."
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti...",13,2,"[Hello, how are you doing?, I love spending ti..."
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...,13,12,[Hi how are you doing? I am okay how about you...
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live...",13,6,"[Hi I dye my hair 4 times a year., Do you live..."
...,...,...,...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...,13,12,[Hello I am a country guy who wants to move to...
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good...",15,8,"[Hello! How are you this evening?, Pretty good..."
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...,15,8,[My parents are teachers and together we love ...


In [169]:
rand_dialogs['target'] = rand_dialogs['cutted_history'].apply(lambda x: x[-1])
rand_dialogs

,utterance_idx,personality,history,history_len,random_len,cutted_history,target
conv_id,,,,,,,
0,7,"[I read twenty books a year., I'm a stunt doub...","[Hello what are doing today?, I am good, I jus...",15,12,"[Hello what are doing today?, I am good, I jus...","Nice, I only have one parent so now I help out..."
1,6,"[I have two dogs., I like to work on vintage c...","[Hi! how are you doing tonight?, I'm doing gre...",13,2,"[Hi! how are you doing tonight?, I'm doing gre...",I'm doing great. Just relaxing with my two dogs.
2,6,"[I love disneyland and mickey mouse., I love t...","[Hello, how are you doing?, I love spending ti...",13,2,"[Hello, how are you doing?, I love spending ti...",I love spending time with my family.
3,6,"[I love to drink fancy tea., I have a big libr...",[Hi how are you doing? I am okay how about you...,13,12,[Hi how are you doing? I am okay how about you...,I've a tough time hearing so reading is my hap...
4,6,"[I'm a little girl., I've superpowers., I like...","[Hi I dye my hair 4 times a year., Do you live...",13,6,"[Hi I dye my hair 4 times a year., Do you live...",I love living in the clouds the best. Do you h...
...,...,...,...,...,...,...,...
995,6,"[My husband is adopted., I've a german shepher...",[Hello I am a country guy who wants to move to...,13,12,[Hello I am a country guy who wants to move to...,Sure. I can not really say the same about law ...
996,7,"[I produce music for artists., I drive a 2015 ...","[Hello! How are you this evening?, Pretty good...",15,8,"[Hello! How are you this evening?, Pretty good...",I produce music. What do you do at the bank?
997,7,"[I'm terrified of scorpions., I'm employed by ...",[My parents are teachers and together we love ...,15,8,[My parents are teachers and together we love ...,"My dog which is a german shepard, is a retired..."


In [170]:
rand_dialogs['cutted_history'][0]

['Hello what are doing today?',
 'I am good, I just got off work and tired, I have two jobs.',
 'I just got done watching a horror movie.',
 "I rather read, I've read about 20 books this year.",
 'Wow! I do love a good horror movie. Loving this cooler weather.',
 'But a good movie is always good.',
 'Yes! My son is in junior high and I just started letting him watch them too.',
 'I work in the movies as well.',
 'Neat!! I used to work in the human services field.',
 'Yes it is neat, I stunt double, it is so much fun and hard work.',
 'Yes I bet you can get hurt. My wife works and I stay at home.',
 'Nice, I only have one parent so now I help out my mom.']

In [171]:
rand_dialogs.to_csv('dialogues_with_random_len.csv')

In [172]:
len_dialogs = pd.read_csv('dialogues_with_len_2_12.csv')
len_dialogs

,conv_id,utterance_idx,personality,history,history_len,random_len,len_2,len_4,len_6,len_8,...,dialogue_8,dialogue_10,dialogue_12,target_2,target_4,target_6,target_8,target_10,target_12,persona
0,0,7,"['I read twenty books a year.', ""I'm a stunt d...","['Hello what are doing today?', 'I am good, I ...",15,12,"['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...","['Hello what are doing today?', 'I am good, I ...",...,HUMAN: Hello what are doing today?\nPERSON: I ...,HUMAN: Hello what are doing today?\nPERSON: I ...,HUMAN: Hello what are doing today?\nPERSON: I ...,"I am good, I just got off work and tired, I ha...","I rather read, I've read about 20 books this y...",But a good movie is always good.,I work in the movies as well.,"Yes it is neat, I stunt double, it is so much ...","Nice, I only have one parent so now I help out...",I read twenty books a year. I'm a stunt double...
1,1,6,"['I have two dogs.', 'I like to work on vintag...","['Hi! how are you doing tonight?', ""I'm doing ...",13,2,"['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...","['Hi! how are you doing tonight?', ""I'm doing ...",...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,HUMAN: Hi! how are you doing tonight?\nPERSON:...,I'm doing great. Just relaxing with my two dogs.,That's neat. What kind of volunteer work do yo...,Good for you. Do you like vintage cars? I've t...,Cars are my thing. Vintage cars. I love workin...,I didn't think about the crowd aspect of wrest...,I agree. Working out is a great way to burn of...,I have two dogs. I like to work on vintage car...
2,2,6,"['I love disneyland and mickey mouse.', 'I lov...","['Hello, how are you doing?', 'I love spending...",13,2,"['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...","['Hello, how are you doing?', 'I love spending...",...,"HUMAN: Hello, how are you doing?\nPERSON: I lo...","HUMAN: Hello, how are you doing?\nPERSON: I lo...","HUMAN: Hello, how are you doing?\nPERSON: I lo...",I love spending time with my family.,So then have you ever been to disneyland?,Yes I love mickey mouse such a cute little rat.,Man your lucky I am a baby delivery nurse and ...,I have to walk 3 miles to work to save money e...,Your economic skills are amazing.,I love disneyland and mickey mouse. I love to ...
3,3,6,"['I love to drink fancy tea.', 'I have a big l...",['Hi how are you doing? I am okay how about yo...,13,12,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,['Hi how are you doing? I am okay how about yo...,...,HUMAN: Hi how are you doing? I am okay how abo...,HUMAN: Hi how are you doing? I am okay how abo...,HUMAN: Hi how are you doing? I am okay how abo...,I am doing good. Just sipping tea. What do you...,I am sorry to hear that. What happened.,So what do you do now for fun? I like to read.,What kind of dogs do you have?,I love reading. I have a big library at my house.,I've a tough time hearing so reading is my hap...,I love to drink fancy tea. I have a big librar...
4,4,6,"[""I'm a little girl."", ""I've superpowers."", 'I...","['Hi I dye my hair 4 times a year.', 'Do you l...",13,6,"['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...","['Hi I dye my hair 4 times a year.', 'Do you l...",...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,HUMAN: Hi I dye my hair 4 times a year.\nPERSO...,Do you live in the sky? I do. in the clouds.,I make time stop. I've a superpower. I'm a sup...,I love living in the

In [175]:
def make_finedeval_form(dial_list):
    new_form = '|||'.join(dial_list[:len(dial_list) - 1])
    return new_form

In [176]:
make_finedeval_form(['Hello what are doing today?',
 'I am good, I just got off work and tired, I have two jobs.',
 'I just got done watching a horror movie.',
 "I rather read, I've read about 20 books this year.",
 'Wow! I do love a good horror movie. Loving this cooler weather.',
 'But a good movie is always good.',
 'Yes! My son is in junior high and I just started letting him watch them too.',
 'I work in the movies as well.',
 'Neat!! I used to work in the human services field.',
 'Yes it is neat, I stunt double, it is so much fun and hard work.',
 'Yes I bet you can get hurt. My wife works and I stay at home.',
 'Nice, I only have one parent so now I help out my mom.'])

"Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather.|||But a good movie is always good.|||Yes! My son is in junior high and I just started letting him watch them too.|||I work in the movies as well.|||Neat!! I used to work in the human services field.|||Yes it is neat, I stunt double, it is so much fun and hard work.|||Yes I bet you can get hurt. My wife works and I stay at home."

In [178]:
fined_rand = rand_dialogs[['cutted_history', 'target']].copy()
fined_rand

,cutted_history,target
conv_id,,
0,"[Hello what are doing today?, I am good, I jus...","Nice, I only have one parent so now I help out..."
1,"[Hi! how are you doing tonight?, I'm doing gre...",I'm doing great. Just relaxing with my two dogs.
2,"[Hello, how are you doing?, I love spending ti...",I love spending time with my family.
3,[Hi how are you doing? I am okay how about you...,I've a tough time hearing so reading is my hap...
4,"[Hi I dye my hair 4 times a year., Do you live...",I love living in the clouds the best. Do you h...
...,...,...
995,[Hello I am a country guy who wants to move to...,Sure. I can not really say the same about law ...
996,"[Hello! How are you this evening?, Pretty good...",I produce music. What do you do at the bank?
997,[My parents are teachers and together we love ...,"My dog which is a german shepard, is a retired..."


In [179]:
fined_rand['tabbed'] = fined_rand['cutted_history'].apply(make_finedeval_form)
fined_rand

,cutted_history,target,tabbed
conv_id,,,
0,"[Hello what are doing today?, I am good, I jus...","Nice, I only have one parent so now I help out...","Hello what are doing today?|||I am good, I jus..."
1,"[Hi! how are you doing tonight?, I'm doing gre...",I'm doing great. Just relaxing with my two dogs.,Hi! how are you doing tonight?
2,"[Hello, how are you doing?, I love spending ti...",I love spending time with my family.,"Hello, how are you doing?"
3,[Hi how are you doing? I am okay how about you...,I've a tough time hearing so reading is my hap...,Hi how are you doing? I am okay how about you?...
4,"[Hi I dye my hair 4 times a year., Do you live...",I love living in the clouds the best. Do you h...,Hi I dye my hair 4 times a year.|||Do you live...
...,...,...,...
995,[Hello I am a country guy who wants to move to...,Sure. I can not really say the same about law ...,Hello I am a country guy who wants to move to ...
996,"[Hello! How are you this evening?, Pretty good...",I produce music. What do you do at the bank?,Hello! How are you this evening?|||Pretty good...
997,[My parents are teachers and together we love ...,"My dog which is a german shepard, is a retired...",My parents are teachers and together we love b...


In [183]:
df_var_len = df_dialogues[['history', 'history_len']].copy()
for i in range(2, 13, 2):
    df_var_len[f'len_{i}'] = df_var_len.apply(lambda x: cut_list(x['history'], i), axis=1)
df_var_len

,history,history_len,len_2,len_4,len_6,len_8,len_10,len_12
conv_id,,,,,,,,
0,"[Hello what are doing today?, I am good, I jus...",15,"[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus..."
1,"[Hi! how are you doing tonight?, I'm doing gre...",13,"[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre..."
2,"[Hello, how are you doing?, I love spending ti...",13,"[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti..."
3,[Hi how are you doing? I am okay how about you...,13,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...
4,"[Hi I dye my hair 4 times a year., Do you live...",13,"[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live..."
...,...,...,...,...,...,...,...,...
995,[Hello I am a country guy who wants to move to...,13,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...
996,"[Hello! How are you this evening?, Pretty good...",15,"[Hello! How are you this evening?, Pretty good...","[Hello! How are you this evening?, Pretty good...","[Hello! How are you this evening?, Pretty good...","[Hello! How are you this evening?, Pretty good...","[Hello! How are you this evening?, Pretty good...","[Hello! How are you this evening?, Pretty good..."
997,[My parents are teachers and together we love ...,15,[My parents are teachers and together we love ...,[My parents are teachers and together we love ...,[My parents are teachers and together we love ...,[My parents are teachers and together we love ...,[My parents are teachers and together we love ...,[My parents are teachers and together we love ...


In [184]:
for i in range(2, 13, 2):
    df_var_len[f'tabbed_{i}'] = df_var_len[f'len_{i}'].apply(make_finedeval_form)
df_var_len

,history,history_len,len_2,len_4,len_6,len_8,len_10,len_12,tabbed_2,tabbed_4,tabbed_6,tabbed_8,tabbed_10,tabbed_12
conv_id,,,,,,,,,,,,,,
0,"[Hello what are doing today?, I am good, I jus...",15,"[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...","[Hello what are doing today?, I am good, I jus...",Hello what are doing today?,"Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus..."
1,"[Hi! how are you doing tonight?, I'm doing gre...",13,"[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...","[Hi! how are you doing tonight?, I'm doing gre...",Hi! how are you doing tonight?,Hi! how are you doing tonight?|||I'm doing gre...,Hi! how are you doing tonight?|||I'm doing gre...,Hi! how are you doing tonight?|||I'm doing gre...,Hi! how are you doing tonight?|||I'm doing gre...,Hi! how are you doing tonight?|||I'm doing gre...
2,"[Hello, how are you doing?, I love spending ti...",13,"[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","[Hello, how are you doing?, I love spending ti...","Hello, how are you doing?","Hello, how are you doing?|||I love spending ti...","Hello, how are you doing?|||I love spending ti...","Hello, how are you doing?|||I love spending ti...","Hello, how are you doing?|||I love spending ti...","Hello, how are you doing?|||I love spending ti..."
3,[Hi how are you doing? I am okay how about you...,13,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,[Hi how are you doing? I am okay how about you...,Hi how are you doing? I am okay how about you?,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...
4,"[Hi I dye my hair 4 times a year., Do you live...",13,"[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...","[Hi I dye my hair 4 times a year., Do you live...",Hi I dye my hair 4 times a year.,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,[Hello I am a country guy who wants to move to...,13,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,[Hello I am a country guy who wants to move to...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who

In [187]:
df_var_len['tabbed_6'][0]

"Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather."

In [188]:
df_var_len.to_csv('tabbed_dialogues_with_len_2_12.csv')
fined_rand.to_csv('tabbed_dialogues_with_random_len.csv')

In [190]:
names1 = ['output_task_pers_dial_random_1.csv',
        'output_zeroshot_random_1.csv',
        'output_pers_dial_random_1.csv',
        'output_cherry_pers_dial_random_1.csv',
        'output_cherry_task_pers_dial_random_1.csv',
        'output_random_eg_pers_dial_1.csv',
        'output_task_random_eg_pers_dial_1.csv'
         ]
names2 = ['output_pers_dial_length_2.csv',
        'output_pers_dial_length_4.csv',
        'output_pers_dial_length_6.csv',
        'output_pers_dial_length_8.csv',
        'output_pers_dial_length_10.csv',
        'output_pers_dial_length_12.csv'
        ]

In [192]:
def concat_pred(tabbed_str, pred):
    return tabbed_str + '|||' + pred

In [197]:
fined_rand['tabbed'][0]

"Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather.|||But a good movie is always good.|||Yes! My son is in junior high and I just started letting him watch them too.|||I work in the movies as well.|||Neat!! I used to work in the human services field.|||Yes it is neat, I stunt double, it is so much fun and hard work.|||Yes I bet you can get hurt. My wife works and I stay at home."

In [212]:
for i in range(len(names1)):
    current_name = names1[i]
    out = pd.read_csv(current_name)
    out.fillna(' ', inplace=True)
    fined_rand[f'pred_{current_name}'] = out['pred'].copy()
    fined_rand[current_name] = fined_rand.apply(lambda x: concat_pred(x['tabbed'], x[f'pred_{current_name}']), axis=1)
fined_rand

,cutted_history,target,tabbed,pred_output_task_pers_dial_random_1.csv,output_task_pers_dial_random_1.csv,pred_output_zeroshot_random_1.csv,output_zeroshot_random_1.csv,pred_output_pers_dial_random_1.csv,output_pers_dial_random_1.csv,pred_output_cherry_pers_dial_random_1.csv,output_cherry_pers_dial_random_1.csv,pred_output_cherry_task_pers_dial_random_1.csv,output_cherry_task_pers_dial_random_1.csv,pred_output_random_eg_pers_dial_1.csv,output_random_eg_pers_dial_1.csv,pred_output_task_random_eg_pers_dial_1.csv,output_task_random_eg_pers_dial_1.csv
conv_id,,,,,,,,,,,,,,,,,
0,"[Hello what are doing today?, I am good, I jus...","Nice, I only have one parent so now I help out...","Hello what are doing today?|||I am good, I jus...","I really do work, I love movies.","Hello what are doing today?|||I am good, I jus...",That is also an option. Yes!,"Hello what are doing today?|||I am good, I jus...",I have two jobs is also neat. My kids are grow...,"Hello what are doing today?|||I am good, I jus...","I have a house, I only use it for storage.","Hello what are doing today?|||I am good, I jus...","Well yes, I love it. I did not get shot once f...","Hello what are doing today?|||I am good, I jus...","Yeah, I was in one where someone was attacked ...","Hello what are doing today?|||I am good, I jus...","I am only 21, and I am a junior in college.","Hello what are doing today?|||I am good, I jus..."
1,"[Hi! how are you doing tonight?, I'm doing gre...",I'm doing great. Just relaxing with my two dogs.,Hi! how are you doing tonight?,Hey. I am fine. Thanks. How about yourself?,Hi! how are you doing tonight?|||Hey. I am fin...,,Hi! how are you doing tonight?|||,I would prefer to keep this a secret.,Hi! how are you doing tonight?|||I would prefe...,Hi how are you doing today?,Hi! how are you doing tonight?|||Hi how are yo...,Hi! I love the outdoors! I'm not the best with...,Hi! how are you doing tonight?|||Hi! I love th...,"Good, enjoying the evening.","Hi! how are you doing tonight?|||Good, enjoyin...",I am a good person. My favorite color is black...,Hi! how are you doing tonight?|||I am a good p...
2,"[Hello, how are you doing?, I love spending ti...",I love spending time with my family.,"Hello, how are you doing?",I’m fine. What about you? I’m glad we finally ...,"Hello, how are you doing?|||I’m fine. What abo...",(Speaking in foreign language.),"Hello, how are you doing?|||(Speaking in forei...",Fine!,"Hello, how are you doing?|||Fine!",Hello. I'm good. How are you doing?,"Hello, how are you doing?|||Hello. I'm good. H...","Oh, just hanging in there. A bit of a cold.","Hello, how are you doing?|||Oh, just hanging i...",I'm doing great. How about you?,"Hello, how are you doing?|||I'm doing great. H...","Nice to meet you, I'm Jane.","Hello, how are you doing?|||Nice to meet you, ..."
3,[Hi how are you doing? I am okay how about you...,I've a tough time hearing so reading is my hap...,Hi how are you doing? I am okay how about you?...,I read to so many I have a hard time finding s...,Hi how are you doing? I am okay how about you?...,So what type of books do you like to read?,Hi how are you doing? I am okay how about you?...,I have a degree in psychology. What about you ...,Hi how are you doing? I am okay how about you?...,I do too. I love reading books.,Hi how are you doing? I am okay how about you?...,I am a little deaf. You are?,Hi how are you doing? I am okay how about you?...,Me too! I listen too.,Hi how are you doing? I am okay how about you?...,I read about dogs. What do you enjoy doing? I ...,Hi how are you doing? I am okay how about you?...
4,"[Hi I dye my hair 4 times a year., Do you live...",I love living in the clouds the best. Do you h...,Hi I dye my hair 4 times a year.|||Do you live...,"I am a very good singer, I've a perfect 10.",Hi I dye my hair 4 times a year.|||Do you live...,You also have a superpower too.,Hi I dye my hair 4 times a year.|||Do you live...,I see you're having fun.,Hi I dye my hair 4 times a year.|||Do you live

In [213]:
fined_rand['output_pers_dial_random_1.csv'][0]

"Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather.|||But a good movie is always good.|||Yes! My son is in junior high and I just started letting him watch them too.|||I work in the movies as well.|||Neat!! I used to work in the human services field.|||Yes it is neat, I stunt double, it is so much fun and hard work.|||Yes I bet you can get hurt. My wife works and I stay at home.|||I have two jobs is also neat. My kids are grown adults."

In [214]:
names1

['output_task_pers_dial_random_1.csv',
 'output_zeroshot_random_1.csv',
 'output_pers_dial_random_1.csv',
 'output_cherry_pers_dial_random_1.csv',
 'output_cherry_task_pers_dial_random_1.csv',
 'output_random_eg_pers_dial_1.csv',
 'output_task_random_eg_pers_dial_1.csv']

In [215]:
finedeval_inputs = fined_rand[names1].copy()
finedeval_inputs

,output_task_pers_dial_random_1.csv,output_zeroshot_random_1.csv,output_pers_dial_random_1.csv,output_cherry_pers_dial_random_1.csv,output_cherry_task_pers_dial_random_1.csv,output_random_eg_pers_dial_1.csv,output_task_random_eg_pers_dial_1.csv
conv_id,,,,,,,
0,"Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus..."
1,Hi! how are you doing tonight?|||Hey. I am fin...,Hi! how are you doing tonight?|||,Hi! how are you doing tonight?|||I would prefe...,Hi! how are you doing tonight?|||Hi how are yo...,Hi! how are you doing tonight?|||Hi! I love th...,"Hi! how are you doing tonight?|||Good, enjoyin...",Hi! how are you doing tonight?|||I am a good p...
2,"Hello, how are you doing?|||I’m fine. What abo...","Hello, how are you doing?|||(Speaking in forei...","Hello, how are you doing?|||Fine!","Hello, how are you doing?|||Hello. I'm good. H...","Hello, how are you doing?|||Oh, just hanging i...","Hello, how are you doing?|||I'm doing great. H...","Hello, how are you doing?|||Nice to meet you, ..."
3,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...
4,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...
...,...,...,...,...,...,...,...
995,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...
996,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...
997,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...


In [15]:
finedeval_inputs.to_csv('finedeval_inputs1.csv')

In [16]:
finedeval_inputs = pd.read_csv('finedeval_inputs1.csv')
finedeval_inputs

,Unnamed: 0,conv_id,output_task_pers_dial_random_1.csv,output_zeroshot_random_1.csv,output_pers_dial_random_1.csv,output_cherry_pers_dial_random_1.csv,output_cherry_task_pers_dial_random_1.csv,output_random_eg_pers_dial_1.csv,output_task_random_eg_pers_dial_1.csv,super_input
0,0,0,"Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...","Hello what are doing today?|||I am good, I jus...",original\tHello what are doing today?|||I am g...
1,1,1,Hi! how are you doing tonight?|||Hey. I am fin...,Hi! how are you doing tonight?|||,Hi! how are you doing tonight?|||I would prefe...,Hi! how are you doing tonight?|||Hi how are yo...,Hi! how are you doing tonight?|||Hi! I love th...,"Hi! how are you doing tonight?|||Good, enjoyin...",Hi! how are you doing tonight?|||I am a good p...,original\tHi! how are you doing tonight?|||I w...
2,2,2,"Hello, how are you doing?|||I’m fine. What abo...","Hello, how are you doing?|||(Speaking in forei...","Hello, how are you doing?|||Fine!","Hello, how are you doing?|||Hello. I'm good. H...","Hello, how are you doing?|||Oh, just hanging i...","Hello, how are you doing?|||I'm doing great. H...","Hello, how are you doing?|||Nice to meet you, ...","original\tHello, how are you doing?|||Fine!\tH..."
3,3,3,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,Hi how are you doing? I am okay how about you?...,original\tHi how are you doing? I am okay how ...
4,4,4,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,Hi I dye my hair 4 times a year.|||Do you live...,original\tHi I dye my hair 4 times a year.|||D...
...,...,...,...,...,...,...,...,...,...,...
995,995,995,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,Hello I am a country guy who wants to move to ...,original\tHello I am a country guy who wants t...
996,996,996,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,Hello! How are you this evening?|||Pretty good...,original\tHello! How are you this evening?|||P...
997,997,997,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,My parents are teachers and together we love b...,original\tMy parents are teachers and together...
998,998,998,"How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...","How are you doing today.|||Doing good man, jus...",original\tHow are you doing today.|||D

In [17]:
def make_super_input(col1, col2):
    return 'original\t' + col1 + '\t' + col2

In [27]:
finedeval_inputs['super_input'] = finedeval_inputs.apply(lambda x: make_super_input(x['output_task_random_eg_pers_dial_1.csv'], x['output_random_eg_pers_dial_1.csv']), axis=1)
finedeval_inputs['super_input']

0      original\tHello what are doing today?|||I am g...
1      original\tHi! how are you doing tonight?|||I a...
2      original\tHello, how are you doing?|||Nice to ...
3      original\tHi how are you doing? I am okay how ...
4      original\tHi I dye my hair 4 times a year.|||D...
                             ...                        
995    original\tHello I am a country guy who wants t...
996    original\tHello! How are you this evening?|||P...
997    original\tMy parents are teachers and together...
998    original\tHow are you doing today.|||Doing goo...
999    original\tHi how are you?|||I am ok. I love th...
Name: super_input, Length: 1000, dtype: object

In [28]:
print(finedeval_inputs['super_input'][0])

original	Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather.|||But a good movie is always good.|||Yes! My son is in junior high and I just started letting him watch them too.|||I work in the movies as well.|||Neat!! I used to work in the human services field.|||Yes it is neat, I stunt double, it is so much fun and hard work.|||Yes I bet you can get hurt. My wife works and I stay at home.|||I am only 21, and I am a junior in college.	Hello what are doing today?|||I am good, I just got off work and tired, I have two jobs.|||I just got done watching a horror movie.|||I rather read, I've read about 20 books this year.|||Wow! I do love a good horror movie. Loving this cooler weather.|||But a good movie is always good.|||Yes! My son is in junior high and I just started letting him watch them too.

In [29]:
with open('out7_out6.txt', 'w') as f:
    for i in range(1000):
        f.write(finedeval_inputs['super_input'][i])
        f.write('\n')

In [225]:
finedeval_inputs['super_input'].to_string(header=False, index=False)

"original\\tHello what are doing today?|||I am go...\noriginal\\tHi! how are you doing tonight?|||Hey....\noriginal\\tHello, how are you doing?|||I’m fine....\noriginal\\tHi how are you doing? I am okay how a...\noriginal\\tHi I dye my hair 4 times a year.|||Do...\noriginal\\tHi! do you like turtles?|||I am much ...\noriginal\\tHello, how are you tonight? Do you ha...\noriginal\\tHi, how are you doing today?|||Good! ...\noriginal\\tHello, are you attending church in th...\noriginal\\tHey, I enjoy mowing lawns in the summ...\noriginal\\tHow are you? I'm tired of my parents ...\noriginal\\tHi. I'm billy. I work as an electrici...\noriginal\\tHello. I love to read and my favorite...\noriginal\\tHello! I was three when I learned to ...\noriginal\\tHello, I'm sitting here with my dog. ...\noriginal\\tHey how are you today.|||Fine. Are yo...\noriginal\\tHello there, how are you today?|||Hel...\noriginal\\tHi, Kevin here. I love mexican food, ...\noriginal\\tHello. My name is rob. How are you

In [30]:
import json

data = pd.read_json('FineD-Eval/output/my_prediction_dailydialog_234567/predictions.fed-small1.end.json')
data

,0,1
0,0.617085,0.624696
1,0.645119,0.633519
2,0.581475,0.522754
3,0.610748,0.619911
4,0.506947,0.569170
...,...,...
995,0.736644,0.755225
996,0.620306,0.620646
997,0.591924,0.612728
998,0.604849,0.602079


In [31]:
statistics.mean(data[0])

0.5887131615579124

In [32]:
statistics.mean(data[1])

0.5830415446907278

In [33]:
import json

data = pd.read_json('FineD-Eval/output/my_prediction_dailydialog_234567/expert_predictions.fed-small1.end.json')
data

,0,1
0,"[0.41556632518768305, 0.634277403354644, 0.801...","[0.5083041787147521, 0.601861596107482, 0.7639..."
1,"[0.7763873934745781, 0.8140925765037531, 0.344...","[0.9230234622955321, 0.7698614597320551, 0.207..."
2,"[0.6475971937179561, 0.822140038013458, 0.2746...","[0.739977836608886, 0.731905162334442, 0.09637..."
3,"[0.6371832489967341, 0.5848175287246701, 0.610...","[0.6937320232391351, 0.605626583099365, 0.5603..."
4,"[0.355282187461853, 0.5631495714187621, 0.6024...","[0.442408770322799, 0.588700890541076, 0.67640..."
...,...,...
995,"[0.802849769592285, 0.7531683444976801, 0.6539...","[0.8431214094161981, 0.766737401485443, 0.6558..."
996,"[0.9164841771125791, 0.6132020354270931, 0.331...","[0.932230710983276, 0.602221667766571, 0.32748..."
997,"[0.507307410240173, 0.34077790379524203, 0.927...","[0.5718754529953001, 0.32636392116546603, 0.93..."
998,"[0.5912985205650331, 0.8577866554260251, 0.365...","[0.604960441589355, 0.842442035675048, 0.35883..."


In [34]:
coherence = []
likability = []
topic_depth = []
for i in range(1000):
    coherence.append(data[0][i][0])
    likability.append(data[0][i][1])
    topic_depth.append(data[0][i][2])
print(statistics.mean(coherence))
print(statistics.mean(likability))
print(statistics.mean(topic_depth))

0.6146736169536594
0.6235535596348342
0.5279122515013437


In [35]:
coherence = []
likability = []
topic_depth = []
for i in range(1000):
    coherence.append(data[1][i][0])
    likability.append(data[1][i][1])
    topic_depth.append(data[1][i][2])
print(statistics.mean(coherence))
print(statistics.mean(likability))
print(statistics.mean(topic_depth))

0.6064252780298698
0.6200985156819221
0.5226007849611337
